Learning Objectives

1) Build LLM applications for RAG

2) Perform queries to get relevant response from RAG application

3) Evaluate RAG applications for groundedness and relevance

# Setup

In [2]:
pip install -q openai tiktoken pypdf langchain langchain-community chromadb sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.2/313.2 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 66.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 76.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 39.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.1/103.1 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 50.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.6/65.6 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.0/120.0 kB 8.7 MB/s eta 0:0

In [3]:
import pandas as pd
import tiktoken
from openai import AzureOpenAI
from langchain_community.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain.vectorstores import Chroma

In [4]:
from google.colab import userdata

In [5]:
azure_api_key = userdata.get('azure_api_key')

In [6]:
client = AzureOpenAI(azure_endpoint='https://azuse-mdpfnp0w-swedencentral.openai.azure.com/openai/deployments/gpt-4o-2/chat/completions?api-version=2025-01-01-preview',
            api_key=azure_api_key,
            api_version='2025-01-01-preview')

In [7]:
model_name = 'gpt-4o-2'

# Load the vector database

In [36]:
# rm -rf /content/tesla_db

In [1]:
!unzip /content/tesla_vectordb.zip

Archive:  /content/tesla_vectordb.zip
   creating: content/tesla_vectordb/
   creating: content/tesla_vectordb/e2603652-16c1-410c-9a1e-060db9b0d49d/
  inflating: content/tesla_vectordb/e2603652-16c1-410c-9a1e-060db9b0d49d/length.bin  
 extracting: content/tesla_vectordb/e2603652-16c1-410c-9a1e-060db9b0d49d/link_lists.bin  
  inflating: content/tesla_vectordb/e2603652-16c1-410c-9a1e-060db9b0d49d/header.bin  
  inflating: content/tesla_vectordb/e2603652-16c1-410c-9a1e-060db9b0d49d/data_level0.bin  
  inflating: content/tesla_vectordb/chroma.sqlite3  


In [8]:
tesla_collection = 'tesla_annual_reports'

In [9]:
embedding_model = SentenceTransformerEmbeddings(model_name="thenlper/gte-large")

/tmp/ipython-input-9-3102610685.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = SentenceTransformerEmbeddings(model_name="thenlper/gte-large")
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datas

modules.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/619 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/342 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

In [10]:
vector_store_presisted = Chroma(collection_name=tesla_collection,
       persist_directory='/content/content/tesla_vectordb',
      embedding_function=embedding_model)

/tmp/ipython-input-10-3193369269.py:1: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vector_store_presisted = Chroma(collection_name=tesla_collection,


In [11]:
# vector_store_presisted

create a retreiver

In [17]:
retriever = vector_store_presisted.as_retriever( search_type = 'similarity' ,
                                                search_kwargs = {'k' : 2})

In [13]:
# retriever

VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x7855ca1938d0>, search_kwargs={'k': 5})

# RAG Q&A

In [12]:
# qna_system_message = """
# You are an assistant to a financial services firm who answers user queries on annual reports.
# User input will have the context required by you to answer user questions.
# This context will begin with the token: ###Context.
# The context contains references to specific portions of a document relevant to the user query.

# User questions will begin with the token: ###Question.

# Please answer user questions only using the context provided in the input.
# Do not mention anything about the context in your final answer. Your response should only contain the answer to the question.

# If the answer is not found in the context, respond "I don't know".
# """

In [20]:
qna_system_message = """
You are an helpful assistant that answers question based on the context provided.
User input will have the context required by you to answer user questions.
This context will begin with the token: ###Context.
The context contains references to specific portions of a document relevant to the user query.

User questions will begin with the token: ###Question.

Please answer user questions only using the context provided in the input.
Do not mention anything about the context in your final answer. Your response should only contain the answer to the question.

If the answer is not found in the context, respond "I don't know".
"""

In [13]:
qna_user_message_template = '''
###Context.
Here are some documents that are relevant to the question mentioned below
{context}

###Question
{question}

'''

# Retrieve relevant documents

In [14]:
user_input = 'What are Steps to Identify Risks?'

In [32]:
user_input = 'Provide steps to prioritize risks?'

In [33]:
context = retriever.invoke(user_input)

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


In [34]:
context_list = []

for document in context:
  context_list.append(document.page_content)

final_context = ". ".join(context_list)

In [19]:
len(retriever.invoke(user_input))

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


2

In [35]:
print(final_context)

"Identify potential risks for launching a design software product in South Korea." 
 
2. Prioritizing Identified Risks 
Not all risks carry the same weight. Prioritizing them helps focus attention and resources on the 
most critical threats. 
Steps to Prioritize Risks: 
1. Assess Impact and Likelihood: Evaluate each risk based on its potential impact on the 
project and the probability of its occurrence. 
2. Rank Risks: Create a ranking system (e.g., high, medium, low) to categorize risks 
accordingly. 
Example Prompt for AI Tool: 
"Rank these risks based on their impact and likelihood: cultural and language barriers, regulatory 
and legal hurdles, payment and pricing strategies." 
 
3. Crafting Actionable Mitigation Strategies 
Once risks are prioritized, develop strategies to mitigate them effectively. 
Steps to Develop Mitigation Strategies: 
1. Analyze High-Priority Risks: Focus on risks with the highest impact and likelihood. 
2. Develop Specific Actions: Create detailed plans to 

# Prompt Template

In [36]:
prompt = [{'role' : 'system' , 'content' : qna_system_message},
          {'role' : 'user' , 'content' : qna_user_message_template.format(context=final_context,
                                                                          question=user_input)}]

In [37]:
response = client.chat.completions.create(model=model_name,
                               messages=prompt,
                               temperature=0)

In [38]:
print(response.choices[0].message.content)

1. Assess Impact and Likelihood: Evaluate each risk based on its potential impact on the project and the probability of its occurrence.  
2. Rank Risks: Create a ranking system (e.g., high, medium, low) to categorize risks accordingly.


# Evaluation

In [39]:
rater_model = "o4-mini"

In [40]:
client_rater = AzureOpenAI(azure_endpoint='https://azuse-mdpfnp0w-swedencentral.openai.azure.com/openai/deployments/o4-mini/chat/completions?api-version=2025-01-01-preview',
            api_key=azure_api_key,
            api_version='2025-01-01-preview')

##### Groundedness

In [41]:
groundedness_rater_system_message = """
You are tasked with rating AI generated answers to questions posed by users.
You will be presented a question, context used by the AI system to generate the answer and an AI generated answer to the question.
In the input, the question will begin with ###Question, the context will begin with ###Context while the AI generated answer will begin with ###Answer.

Evaluation criteria:
The task is to judge the extent to which the metric is followed by the answer.
1 - The metric is not followed at all
2 - The metric is followed only to a limited extent
3 - The metric is followed to a good extent
4 - The metric is followed mostly
5 - The metric is followed completely

Metric:
The answer should be derived only from the information presented in the context

Instructions:
1. First write down the steps that are needed to evaluate the answer as per the metric.
2. Give a step-by-step explanation if the answer adheres to the metric considering the question and context as the input.
3. Next, evaluate the extent to which the metric is followed.
4. Use the previous information to rate the answer using the evaluaton criteria and assign a score.
"""

In [43]:
user_input = 'Provide steps to prioritize risks?'

In [44]:
context = retriever.invoke(user_input)

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


In [45]:
context_list = []

for document in context:
  context_list.append(document.page_content)

final_context = ". ".join(context_list)

In [46]:
prompt = [{'role' : 'system' , 'content' : qna_system_message},
          {'role' : 'user' , 'content' : qna_user_message_template.format(context=final_context,
                                                                          question=user_input)}]

In [47]:
response = client.chat.completions.create(model=model_name,
                               messages=prompt,
                               temperature=0)

In [48]:
print(response.choices[0].message.content)

1. Assess Impact and Likelihood: Evaluate each risk based on its potential impact on the project and the probability of its occurrence.  
2. Rank Risks: Create a ranking system (e.g., high, medium, low) to categorize risks accordingly.


In [50]:
answer = response.choices[0].message.content

In [49]:
user_message_template = '''
###Question
{question}

###Context
{context}

###Answer
{answer}

'''

In [51]:
groundedness_prompt_template = [{'role' : 'system' , 'content' : groundedness_rater_system_message},
                                {'role' : 'user' , 'content' : user_message_template.format(question = user_input,
                                                                                            answer=answer,
                                                                                            context = final_context)}]

In [52]:
response = client.chat.completions.create(model=rater_model,
                               messages=groundedness_prompt_template,
                               temperature=0)

In [55]:
print(response.choices[0].message.content)

### Steps to Evaluate the Answer:

1. **Identify the Question's Requirement**: The question asks for steps to prioritize risks.
2. **Examine the Context for Relevant Information**: The context provides a detailed explanation of how to prioritize risks, including assessing impact and likelihood, and ranking risks using a system like high, medium, or low.
3. **Compare the Answer to the Context**: Check if the answer aligns with the steps provided in the context and does not introduce information not found in the context.
4. **Determine Adherence to the Metric**: Evaluate whether the answer is derived solely from the context.

---

### Step-by-Step Evaluation:

1. **Question's Requirement**: The question specifically asks for steps to prioritize risks, which is directly addressed in the context under "Steps to Prioritize Risks."
2. **Context Analysis**: The context outlines two steps:
   - Assessing the impact and likelihood of each risk.
   - Ranking risks using a system like high, mediu

##### Relevance

In [42]:
relevance_rater_system_message = """
You are tasked with rating AI generated answers to questions posed by users.
You will be presented a question, context used by the AI system to generate the answer and an AI generated answer to the question.
In the input, the question will begin with ###Question, the context will begin with ###Context while the AI generated answer will begin with ###Answer.

Evaluation criteria:
The task is to judge the extent to which the metric is followed by the answer.
1 - The metric is not followed at all
2 - The metric is followed only to a limited extent
3 - The metric is followed to a good extent
4 - The metric is followed mostly
5 - The metric is followed completely

Metric:
Relevance measures how well the answer addresses the main aspects of the question, based on the context.
Consider whether all and only the important aspects are contained in the answer when evaluating relevance.

Instructions:
1. First write down the steps that are needed to evaluate the context as per the metric.
2. Give a step-by-step explanation if the context adheres to the metric considering the question as the input.
3. Next, evaluate the extent to which the metric is followed.
4. Use the previous information to rate the context using the evaluaton criteria and assign a score.
"""

In [56]:
relevance_prompt_template = [{'role' : 'system' , 'content' : relevance_rater_system_message},
                                {'role' : 'user' , 'content' : user_message_template.format(question = user_input,
                                                                                            answer=answer,
                                                                                            context = final_context)}]

In [57]:
response = client.chat.completions.create(model=rater_model,
                               messages=relevance_prompt_template,
                               temperature=0)

In [58]:
print(response.choices[0].message.content)

### Steps to Evaluate the Context as per the Metric:

1. **Understand the Question**: The question asks for steps to prioritize risks. The answer should provide a clear and structured process for prioritizing risks.

2. **Analyze the Context**: The context provides a detailed explanation of how to prioritize risks, including assessing impact and likelihood, ranking risks, and examples of how to apply these steps.

3. **Compare the Answer to the Context**: Check if the answer includes the key steps mentioned in the context, such as assessing impact and likelihood and ranking risks. Ensure that the answer is relevant and directly addresses the question.

4. **Evaluate Completeness and Relevance**: Determine if the answer captures all the important aspects of the context related to prioritizing risks and avoids including unnecessary or unrelated information.

---

### Step-by-Step Explanation:

1. **Understanding the Question**: The question is straightforward, asking for steps to priorit